In [3]:
import numpy as np 
import unidecode
import pandas as pd 
import re
import nltk 
import string
from nltk.corpus import stopwords
from nltk.stem.rslp import RSLPStemmer


import warnings
warnings.filterwarnings('ignore')

lemmatizer = RSLPStemmer()

In [5]:
import pyspark 
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.1.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.memory","12G") \
    .config("spark.driver.memory","4G") \
    .config("spark.executor.cores","6") \
    .getOrCreate()

In [8]:
dataframe = spark.read.options(
    inferSchema='True',
    delimiter=',',
    header='True').csv("/home/daholive/Documents/twitter_ellection_brazil/datasource/TweetsWithTheme.csv")

In [11]:
dataframe.show(5)

+-------------------+--------------------+--------------------+--------------------+--------------------+
|                 id|          tweet_text|          tweet_date|           sentiment|          query_used|
+-------------------+--------------------+--------------------+--------------------+--------------------+
|1026606627041026050|ela vendo a famíl...|Mon Aug 06 23:11:...|            Negativo|       vera lucia :(|
|1026266117646561281|Eu odeio o fato q...|Mon Aug 06 00:38:...|            Negativo|             ciro :(|
|1025889796097564672|"Ciro ""mita"" ta...| não quero ser ob...|Sat Aug 04 23:42:...|            Negativo|
|1026960047937646598|Raiva do Bolsonar...|Tue Aug 07 22:35:...|            Positivo|        bolsonaro :)|
|1026447105932623872|"Reflexos da alie...| Lula é um corrup...| Haddad é um post...| sem sal e inexpr...|
+-------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [5]:
dataframe = pd.read_csv("/home/daholive/Documents/twitter_ellection_brazil/datasource/TweetsWithTheme.csv", sep=",")

In [6]:
dataframe["sentiment"] = dataframe["sentiment"].map({
    "Negativo": 0,
    "Positivo": 1    
})

In [7]:
def clean_text(texto):
    
    punct = string.punctuation # Cria uma tabela de tradução
    trantab = str.maketrans(punct, len(punct)*' ') # Todo simbolo da pontuação e substituido por um espaço
    
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030""]+", re.UNICODE)
    
    texto = texto.lower()
    texto = re.sub('\d+', '', str(texto)).replace("&gt;"," ").replace("&lt;"," ") 
    texto = re.sub(r"https?:\/\/\S+","", texto)
    texto = re.sub(r"@[A-Za-z0-9\w]+","", texto)
    texto = re.sub(r"#[A-Za-z0-9\w]+","", texto)
    texto = re.sub('^RT ',' ',texto)
    texto = texto.translate(trantab).replace("\n"," ")
    # texto = re.sub(emoj, '', texto).replace("“"," ").replace("”"," ").strip().lower()
    texto = texto.replace("“"," ").replace("”"," ")
    texto = unidecode.unidecode(texto)
    texto = ' '.join([word for word in texto.split() if word not in list(set(stopwords.words("portuguese")))])
    texto = ' '.join([word for word in texto.split() if word.isalnum()])
    texto = ' '.join([re.sub(r'([a-z])\1+', r'\1',word) for word in texto.split()])
    texto = ' '.join([re.sub(r'(ha)\1+', r'\1',word) for word in texto.split()])
    texto = ' '.join([re.sub(r'(uha)\1+', r'\1',word) for word in texto.split()])
    texto = ' '.join([lemmatizer.stem(word) for word in texto.split()])

    return texto.strip()

In [ ]:
features = dataframe["tweet_text"].map(clean_text).values
labels = dataframe["sentiment"].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [ ]:
# Bigram Counts
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features = 1000)
bigram_vectorizer.fit(features)
X_train_bigram = bigram_vectorizer.transform(features)

In [ ]:
# Bigram Tf-Idf
bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)
X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)

In [ ]:
count_vect_df = pd.DataFrame(X_train_bigram_tf_idf.todense(), columns=bigram_vectorizer.get_feature_names())
count_vect_df

In [ ]:
# df_tfidf = pd.DataFrame(
#    processed_features.toarray(), 
#    columns=vectorizer.get_feature_names(), 
#    index=dataframe.index)

In [ ]:
df_setup = pd.concat(
    [dataframe["sentiment"], count_vect_df], 
    axis = 1)
df_setup

In [ ]:
from pycaret.classification import *
# setup = setup(data=df_setup, target='target_cat', session_id=123, train_size = 0.7, fold=10, silent=True)

In [ ]:
pce_1 = setup(
    data = df_setup, 
    target = 'sentiment', 
    session_id = 5, 
    train_size = 0.75,
    fold=5,
    use_gpu = True,
    n_jobs=-1,
    fix_imbalance = True,
    remove_outliers = True,
    handle_unknown_categorical = True, 
    unknown_categorical_method = 'most_frequent')

In [ ]:
best = compare_models(sort='F1')  

# fold=5
#  0.7637
# include=['gbc','lda','ridge','ada','lr','lightgbm'],

In [ ]:
best

"""
RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=5, solver='auto',
                tol=0.001)
"""

In [ ]:
tuned_dt = tune_model(best, fold=5)

In [ ]:
plot_model(tuned_dt, plot = 'auc')

In [ ]:
plot_model(tuned_dt, plot = 'pr')

In [ ]:
plot_model(tuned_dt, plot='feature')

In [ ]:
plot_model(tuned_dt, plot = 'confusion_matrix')

In [ ]:
predict_model(tuned_dt);

In [ ]:
final_rf = finalize_model(tuned_dt)

In [ ]:
save_model(final_rf,'Final Linear Discriminant Analysis')

In [ ]:
saved_final_rf = load_model('Final Linear Discriminant Analysis')

In [ ]:
saved_final_rf

In [ ]:
texto = ["Bolsonaro você é um idiota e vai perder a eleição","Lula é o melhor, e ganha no primeiro turno"]

features = [clean_text(w) for w in texto]
features

In [ ]:
vectorizer = TfidfVectorizer(
    min_df=0.004,
    max_df=0.7
)

In [ ]:
processed_features = vectorizer.fit_transform(features)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
df_tfidf_v2 = pd.DataFrame(
    processed_features.toarray(), 
    columns=vectorizer.get_feature_names())

df_tfidf_v2

In [ ]:

unseen_predictions = predict_model(tuned_dt, data=df_tfidf_v2)
